In [1]:
%load_ext autoreload
%autoreload 2
%run A_simple_net.ipynb

Detected PyNN version 0.9.4 and Neo version 0.6.1
2020-12-04T14:35:26+00:00

CPython 3.6.9
IPython 7.15.0

numpy 1.19.0
matplotlib 3.2.2
pyNN 0.9.4
neo 0.6.1
nixio 1.4.9
pandas 1.1.4

compiler   : GCC 8.4.0
system     : Linux
release    : 4.4.0-87-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit
host name  : 96c3276e2566
Git hash   : e60c9e998152caa924eba8224540e6750edb315d
Git repo   : https://github.com/SpikeAI/2020-11_brainhack_Project7
Git branch : main


In [2]:
%mkdir -p outputs

In [3]:
%mkdir -p Results/20201204/

# scanning over single parameters

## initializing parameters

In [4]:
tag = '2020-12-04_scan_' + simulator

In [5]:
buildCPUTime, simCPUTime, writeCPUTime = run(verbose=True)

2020-12-04 14:35:27 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2020-12-04 14:35:27 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2020-12-04 14:35:27 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2020-12-04 14:35:27 INFO: Setting time scale factor to 10.
2020-12-04 14:35:27 INFO: Setting machine time step to 100 micro-seconds.
2020-12-04 14:35:27 WARNING: Extra params {'threads': 1, 'filename': 'va.xml', 'label': 'VA'} have been a

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']
0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...
0 Running simulation...


2020-12-04 14:35:30 INFO: Time 0:00:03.389558 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 14:35:40 INFO: Time 0:00:09.593805 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-04 14:35:45 INFO: Time 0:00:05.544348 taken by PartitionAndPlacePartitioner
Created spalloc job 5540651
2020-12-04 14:35:45 INFO: Created spalloc job 5540651
Waiting for board power commands to complete.
2020-12-04 14:35:45 INFO: Waiting for board power commands to complete.
2020-12-04 14:35:50 INFO: Time 0:00:05.046543 taken by SpallocAllocator
2020-12-04 14:35:50 INFO: Creating transceiver for 10.11.193.65
2020-12-04 14:35:50 INFO: Working out if machine is booted
2020-12-04 14:35:54 INFO: Attempting to

0 Writing data to file...



Getting v for All Cells
|0%                          50%                         100%|


--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 1000
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 0.88375 Hz
Inhibitory rate        : 0.995 Hz
Build time             : 6.30522 ms
Simulation time        : 66778 ms
Writing time           : 238.363 ms


## Running Model and Loading Results

In [6]:
import os
import pandas as pd
verbose = True
def scan(variable, values, tag=tag, verbose=verbose):
    filename = 'outputs/{tag}_{variable}.json'.format(tag=tag, variable=variable)
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=[variable, 'buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, value in enumerate(values):
            if verbose: print('{variable=}---{value=}'.format(value=value, variable=variable))
            args = {}
            args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= {variable: value, 'buildCPUTime':buildCPUTime, 'simCPUTime':simCPUTime, 'writeCPUTime':writeCPUTime}
        df.to_json(filename)
    return df

In [7]:
import matplotlib.pyplot as plt

# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression
def plot(df, do_intercept=False):
    variable = df.columns[0]
    fig, ax = plt.subplots(figsize=(8,8))
    
    for regressor, color in zip(['buildCPUTime', 'simCPUTime', 'writeCPUTime'],
                                ['r', 'g', 'b']):

        import patsy
        if do_intercept:
            y, X = patsy.dmatrices('{regressor} ~ {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')
        else:
            y, X = patsy.dmatrices('{regressor} ~ 0 + {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')

        import statsmodels.api as sm
        fit = sm.OLS(y, X).fit()
        print(fit.summary())

        if do_intercept: intercept = fit.params.Intercept
        slope = fit.params[variable]

        values = np.array(df[variable])
        if do_intercept:
            ax.plot(values, intercept + slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable} + {intercept:.3f}")
        else:
            ax.plot(values, slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable}")
        df.plot(x=variable, y=regressor, c=color, lw=1, ls='-.', ax=ax, legend=False)

    ax.set_xlim(0)
    ax.set_ylabel('Time (ms)')
    ax.set_ylim(0)
    ax.legend()
    return fig, ax

## plot as a function of neuron numbers

In [8]:
import numpy as np
N_pops = np.array([int(k)**2 for k in np.linspace(8**2, 45**2, 14, endpoint=True)**.5])
N_pops

array([  64,  196,  361,  484,  625,  784,  961, 1089, 1225, 1369, 1521,
       1681, 1849, 2025])

In [9]:
results_pop = scan('N_pop', N_pops)

Loaded file


In [10]:
fig, ax = plot(results_pop)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

                                 OLS Regression Results                                
Dep. Variable:           buildCPUTime   R-squared (uncentered):                   0.729
Model:                            OLS   Adj. R-squared (uncentered):              0.708
Method:                 Least Squares   F-statistic:                              34.95
Date:                Fri, 04 Dec 2020   Prob (F-statistic):                    5.13e-05
Time:                        14:36:34   Log-Likelihood:                         -35.146
No. Observations:                  14   AIC:                                      72.29
Df Residuals:                      13   BIC:                                      72.93
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


In [11]:
fig.savefig(tag + '_N_pop.png')

## plot as a function of simtime

In [12]:
simtimes = np.linspace(500, 3000, 15, endpoint=True)
simtimes

array([ 500.        ,  678.57142857,  857.14285714, 1035.71428571,
       1214.28571429, 1392.85714286, 1571.42857143, 1750.        ,
       1928.57142857, 2107.14285714, 2285.71428571, 2464.28571429,
       2642.85714286, 2821.42857143, 3000.        ])

In [13]:
results_simtime = scan('simtime', simtimes)

Loaded file


In [14]:
fig, ax = plot(results_simtime, do_intercept=True)      

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

                            OLS Regression Results                            
Dep. Variable:           buildCPUTime   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.070
Method:                 Least Squares   F-statistic:                   0.08426
Date:                Fri, 04 Dec 2020   Prob (F-statistic):              0.776
Time:                        14:36:34   Log-Likelihood:                -3.8948
No. Observations:                  15   AIC:                             11.79
Df Residuals:                      13   BIC:                             13.21
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.0068      0.216     23.213      0.0

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


In [15]:
fig.savefig(tag + '_simtime.png')

# scanning over two parameters

## Setting-up model and initializing parameters

In [16]:
tag = '2020-12-04_scan-2D_' + simulator

## Running Model and Loading Results

In [17]:
variables = ['N_pop', 'simtime']
filename = 'outputs/' + tag
for variable in variables: filename +=  '_' + variable
filename += '.json'

In [18]:
import os
import time
import pandas as pd
verbose = True
def scan(variables, values, tag=tag, verbose=verbose):
    filename = 'outputs/' + tag
    for variable in variables: filename +=  '_' + variable
    filename += '.json'
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, values_ in enumerate(values):
            if verbose: print('{variable}---{values_}'.format(values_=values_, variable=variable))
            args = {}
            for variable, value in zip(variables, values_):
                args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= dict(zip(variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'], 
                                list(values_) + [buildCPUTime, simCPUTime, writeCPUTime]))

        df.to_json(filename)
        
    return df

## Running time as a function of neuron numbers and simtime

In [19]:
import numpy as np

In [20]:
N_trials = 200
N_pops = [int(k)**2 for k in np.linspace(15, 60, N_trials, endpoint=True)]
simtimes = np.linspace(200, 3000, N_trials, endpoint=True)

In [21]:
np.random.seed(42)
np.random.shuffle(N_pops)
np.random.shuffle(simtimes)
values = zip(N_pops, simtimes)

In [ ]:
results = scan(variables=variables, values=values)
results

2020-12-04 14:36:35 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2020-12-04 14:36:35 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2020-12-04 14:36:35 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2020-12-04 14:36:35 INFO: Setting time scale factor to 10.
2020-12-04 14:36:35 INFO: Setting machine time step to 100 micro-seconds.
2020-12-04 14:36:35 WARNING: Extra params {'threads': 1, 'filename': 'va.xml', 'label': 'VA'} have been a

simtime---(1296, 2676.381909547739)
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2020-12-04 14:36:38 INFO: Time 0:00:03.503989 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 14:36:47 INFO: Time 0:00:09.141106 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-04 14:36:53 INFO: Time 0:00:05.207506 taken by PartitionAndPlacePartitioner
Created spalloc job 5540652
2020-12-04 14:36:53 INFO: Created spalloc job 5540652
Waiting for board power commands to complete.
2020-12-04 14:36:53 INFO: Waiting for board power commands to complete.
2020-12-04 14:36:58 INFO: Time 0:00:05.045330 taken by SpallocAllocator
2020-12-04 14:36:58 INFO: Creating transceiver for 10.11.193.65
2020-12-04 14:36:58 INFO: Working out if machine is booted
2020-12-04 14:37:02 INFO: Attempting to

simtime---(324, 664.321608040201)
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2020-12-04 14:38:02 INFO: Time 0:00:03.348040 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 14:38:11 INFO: Time 0:00:09.197398 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-04 14:38:16 INFO: Time 0:00:05.225188 taken by PartitionAndPlacePartitioner
Created spalloc job 5540653
2020-12-04 14:38:16 INFO: Created spalloc job 5540653
Waiting for board power commands to complete.
2020-12-04 14:38:17 INFO: Waiting for board power commands to complete.
2020-12-04 14:38:21 INFO: Time 0:00:05.050811 taken by SpallocAllocator
2020-12-04 14:38:21 INFO: Creating transceiver for 10.11.193.65
2020-12-04 14:38:22 INFO: Working out if machine is booted
2020-12-04 14:38:26 INFO: Attempting to

...

2020-12-04 14:44:31 INFO: Time 0:00:02.876006 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 14:44:40 INFO: Time 0:00:09.298040 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-04 14:44:45 INFO: Time 0:00:05.356034 taken by PartitionAndPlacePartitioner
Created spalloc job 5540660
2020-12-04 14:44:45 INFO: Created spalloc job 5540660
Waiting for board power commands to complete.
2020-12-04 14:44:46 INFO: Waiting for board power commands to complete.
2020-12-04 14:44:50 INFO: Time 0:00:05.049607 taken by SpallocAllocator
2020-12-04 14:44:50 INFO: Creating transceiver for 10.11.193.65
2020-12-04 14:44:50 INFO: Working out if machine is booted
2020-12-04 14:44:54 INFO: Attempting to

simtime---(2809, 1269.3467336683418)
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2020-12-04 14:45:42 INFO: Time 0:00:03.967807 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 14:45:52 INFO: Time 0:00:09.683935 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-04 14:45:57 INFO: Time 0:00:05.315642 taken by PartitionAndPlacePartitioner
Created spalloc job 5540662
2020-12-04 14:45:57 INFO: Created spalloc job 5540662
Waiting for board power commands to complete.
2020-12-04 14:45:57 INFO: Waiting for board power commands to complete.
2020-12-04 14:46:02 INFO: Time 0:00:05.047859 taken by SpallocAllocator
2020-12-04 14:46:02 INFO: Creating transceiver for 10.11.193.65
2020-12-04 14:46:02 INFO: Working out if machine is booted
2020-12-04 14:46:06 INFO: Attempting to

simtime---(2916, 1114.572864321608)
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2020-12-04 14:46:55 INFO: Time 0:00:02.815537 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-04 14:47:06 INFO: Time 0:00:10.613916 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
 

## analysis

In [ ]:
results_ext = results.copy()
results_ext['simtime_N_pop'] = results_ext['simtime'] * results_ext['N_pop']
results_ext

In [ ]:
variables_ext = ['N_pop', 'simtime_N_pop']
X = results_ext[variables_ext].values.reshape(-1, len(variables_ext))


https://aegis4048.github.io/mutiple_linear_regression_and_visualization_in_python

See https://patsy.readthedocs.io/en/latest/quickstart.html

In [ ]:
import patsy
import statsmodels.api as sm

for regressor in ['buildCPUTime', 'simCPUTime', 'writeCPUTime']:
    y = results_ext[regressor].values
    #print(variables_ext, X.shape, y.shape)
    y, X = patsy.dmatrices('{regressor} ~ 0 + N_pop + simtime_N_pop'.format(regressor=regressor), data=results_ext, return_type='dataframe')

    fit = sm.OLS(y, X).fit()
    fit.summary()

    print('{regressor} (ms) = {fit0:.3f} * N_pop + {fit1:.3f}/1000 * simtime (ms) * N_pop'.format(regressor=regressor, fit0=fit.params[0], fit1=fit.params[1]*1000))